In [7]:
import pandas as pd
import numpy as np
import os
import folium
from folium import plugins
import geopandas

In [8]:
ROOT = "C:/Users/Beau/Documents/GitHub/RealEstate"
RAW = os.path.join(ROOT, "data", "raw")
SERIALIZED = os.path.join(ROOT, "data", "serialized")
CLEANED = os.path.join(ROOT, "data", "processed")

In [9]:
# serialize data
# path = os.path.join(RAW, "Business_Licenses_-_Current_Liquor_and_Public_Places_of_Amusement_Licenses.csv")
# df = pd.read_csv(path)
# df.to_pickle(os.path.join(SERIALIZED, "Business_Licenses_-_Current_Liquor_and_Public_Places_of_Amusement_Licenses.pkl"))

In [10]:
df = pd.read_pickle(
    os.path.join(
        SERIALIZED,
        "Business_Licenses_-_Current_Liquor_and_Public_Places_of_Amusement_Licenses.pkl",
    )
)

In [12]:
# format dates
df["DATE ISSUED"] = pd.to_datetime(df["DATE ISSUED"])

In [13]:
# drop unnecessary columns
df.drop(
    [
        "ID",
        "ACCOUNT NUMBER",
        "SITE NUMBER",
        "CITY",
        "STATE",
        "PAYMENT DATE",
        "CONDITIONAL APPROVAL",
        "WARD PRECINCT",
        "LICENSE CODE",
        "BUSINESS ACTIVITY ID",
        "APPLICATION CREATED DATE",
        "LICENSE APPROVED FOR ISSUANCE",
        "LICENSE STATUS CHANGE DATE",
        "LICENSE TERM EXPIRATION DATE",
        "APPLICATION REQUIREMENTS COMPLETE",
        "LICENSE STATUS",
        "LOCATION",
    ],
    axis=1,
    inplace=True,
)

In [14]:
# filter out irrelevant rows
df = df.loc[df["LATITUDE"].notnull()]
df = df.loc[df["LONGITUDE"].notnull()]

# limit to food licenses
# df.loc[df['LICENSE DESCRIPTION'].str.contains("Food")].shape

In [25]:
df = df.loc[
    df["LICENSE DESCRIPTION"].isin(
        [
            "Consumption on Premises - Incidental Activity",
            "Tavern",
            "Outdoor Patio",
            "Late Hour",
            "Not-For-Profit Club",
            "Music and Dance",
        ]
    )
]

(4273, 17)

In [24]:
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

geometry = geopandas.points_from_xy(df.LONGITUDE, df.LATITUDE)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in geometry]
plugins.HeatMap(heat_data, radius=5, blur=3).add_to(m)

m

In [26]:
# save to processed folder
df.to_pickle(
    os.path.join(ROOT, "data", "processed", "Business_Licenses_Current_Alcohol.pkl")
)